In [ ]:
%load_ext autoreload
%autoreload 2

### Description

Generate a conditional output distribution according to some constraints

In [ ]:
import numpy as np
import function_generation
import probability_distributions

In [ ]:
#distribution parameters
input_variables = 4
number_of_states = 5
nudge_size = 0.01

#generate both input and conditional output with Dirichlet weights
distribution_shape = [number_of_states]*input_variables
total_number_of_states = reduce(lambda x,y: x*y, distribution_shape)
input_dist = np.random.dirichlet([1]*total_number_of_states)
input_dist = np.reshape(input_dist, distribution_shape)
cond_shape = [number_of_states]*(input_variables+1)
cond_output = [
    probability_distributions.compute_joint_uniform_random((number_of_states,))
    for i in range(number_of_states**(input_variables))
]
cond_output = np.array(cond_output)
cond_output = np.reshape(cond_output, cond_shape)


In [ ]:
PATH = "/home/derkjan/Documents/academics_UVA/master_thesis/code/"
INPUT_FOLDER = "input_distributions/"
COND_OUTPUT_FOLDER = "conditional_output_distributions/"
FOLDER_FORMAT_INPUT = "{}var_{}states/"
FOLDER_FORMAT_CONDITIONAL = "{}var_{}states/"
FILE_FORMAT_INPUT = "dist_{}.npy"
FILE_FORMAT_COND_OUTPUT = "cond_dist_{}.npy"

DIRICHLET_FOLDER_INPUT = "dirichlet/"
ENTROPY_LOW_FOLDER_INPUT = "entropy_0.5/"
ENTROPY_MEDIUM_FOLDER_INPUT = "entropy_0.75/"

DIRICHLET_FOLDER_COND_OUTPUT = "dirichlet/"


def generate_distributions(path_to_files, file_format, number_of_distributions):
    for i in range(number_of_distributions):
        file_name = path_to_files + file_format.format(i)
        with open(file_name, 'rb') as f:
            yield np.load(f)

def generate_distributions_smart(path_to_files, file_format, start_dist, end_dist):
    for i in range(start_dist, end_dist, 1):
        file_name = path_to_files + file_format.format(i)
        #print(file_name)
        with open(file_name, 'rb') as f:
            yield np.load(f)
            
def get_input_dist_entropy_75(number_of_dists, number_of_var, start, end):
    number_of_states = 5
    path_to_input_files = (
        PATH + INPUT_FOLDER + ENTROPY_MEDIUM_FOLDER_INPUT
        + FOLDER_FORMAT_INPUT.format(number_of_var, number_of_states)
    )
    input_generator = generate_distributions_smart(
        path_to_input_files, FILE_FORMAT_INPUT, 
        start, end
    )
    input_shape = [number_of_states]*number_of_var
    input_dists = []
    for sample in range(number_of_dists):
        input_dist = next(input_generator)
        input_dists.append(np.reshape(input_dist, input_shape))
        
    return input_dists


In [ ]:
number_of_vars = 4
number_of_input_dists = 100
input_dists = get_input_dist_entropy_75(
    number_of_input_dists, number_of_vars, 150, 250
)
individual = function_generation.ConditionalOutput(cond_output, start_mutation_size=0.01, change_mutation_size=0.001)
individual.evaluate(input_dists=input_dists, goal_distance=0)
individual.score

In [ ]:
evolutionary_parameters = {
    "number_of_generations": 25,
    "population_size": 10,
    "number_of_children": 20, 
    "generational": True,
    "mutation_size": 2.5,
    "change_mutation_size": 0.5,
    "parent_selection_mode": "rank_exponential",
    "number_of_input_distributions": 200
}
start_dist = 200
end_dist = 400
number_of_vars = 1
number_of_input_dists = 200
input_dists = get_input_dist_entropy_75(
    number_of_input_dists, number_of_vars, start_dist, end_dist
)
input_shape = number_of_vars * [5]
number_of_states_output = 5
goal_distance = 0.3

max_cond_output = function_generation.get_cond_output_with_max_distance(
    input_shape, number_of_states_output, goal_distance, 
    evolutionary_parameters, input_dists
)


In [ ]:
print(max_cond_output.mutation_size)
print(max_cond_output.cond_output)


In [ ]:

mutated_distribution = function_generation.mutate_conditional_distribution_uniform(
    np.copy(cond_output), 0.01
)